## Import Module

In [1]:
import struct
import numpy as np
import pandas as pd
# import import_ipynb
from importnb import Notebook
with Notebook(): 
    import NN_HW3_sm as NN
    import CNN_pool_class as CNN
from tqdm import tqdm, trange, notebook
from matplotlib import pyplot as plt
np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Read Data Function

In [2]:
def read_data(image_dir, label_dir):
    with open(image_dir, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        
        buf = file.read(size * rows * cols)
        img_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float64)
        img_data = img_data.reshape(size, 1, rows, cols)
        img_data = img_data / 255
        
    
    with open(label_dir, 'rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        
        buf = file.read(size)
        lab_data_tmp = np.frombuffer(buf, dtype=np.uint8).astype(np.uint16)
        lab_data_tmp = lab_data_tmp.reshape(size)
        
#     print(lab_data[0:111])
    lab_data = np.zeros((size, 10), dtype=np.uint16)
    for i in range(size):
        lab_data[i][ lab_data_tmp[i] ] = 1
    
#     print(lab_data[0:10])
    
    
    return img_data.copy(), lab_data.copy()

In [3]:
# print(np.frombuffer(b'\x02\x01\x00\x00', dtype=np.uint16))
train_data, train_label  = read_data('./MNIST/train-images-idx3-ubyte', './MNIST/train-labels-idx1-ubyte')
test_data, test_label  = read_data('./MNIST/t10k-images-idx3-ubyte', './MNIST/t10k-labels-idx1-ubyte')

In [4]:
print((train_data[0]))

[[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
  

## Dataset Label

In [5]:
def preprocess_input(x):
    return x

ans = preprocess_input(train_data[0])
ans1 = preprocess_input(test_data[1])
print(ans)
print(ans1)
# print(train_data[0:10])

[[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
  

In [12]:

def construct_model(model):
    #### First Convolution + pooling
    # func, d_func, last_layer, input_size, input_num, filter_size, filter_num, stride
#     model.layer_list.append( CNN.ConvLayer(NN.Sigmoid, NN.d_Sigmoid, 1, input_size=28, input_num=1, filter_size=4, filter_num=4, stride=2, is_first=True))
    # last_layer, input_size, input_num, filter_size, stride
#     model.layer_list.append( CNN.AvgPooling(model.layer_list[0], input_size=13, input_num=4, filter_size=5, stride=2))   
    
    #### Second Convolution + pooling
    # func, d_func, last_layer, input_size, input_num, filter_size, filter_num, stride
#     model.layer_list.append( CNN.ConvLayer(NN.ReLU, NN.d_ReLU, model.layer_list[1], input_size=9, input_num=4, filter_size=3, filter_num=5, stride=1, is_first=False))
    model.layer_list.append( CNN.ConvLayer(NN.ReLU, NN.d_ReLU, 1, input_size=28, input_num=1, filter_size=3, filter_num=5, stride=1, is_first=True))
    # last_layer, input_size, input_num, filter_size, stride
    model.layer_list.append( CNN.AvgPooling(model.layer_list[len(model.layer_list)-1], input_size=model.layer_list[len(model.layer_list)-1].get_output_size(), input_num=5, filter_size=5, stride=1, is_first=False))       
#     model.layer_list.append( CNN.AvgPooling(1, input_size=28, input_num=1, filter_size=3, stride=1, is_first=True))       
    
    # last_layer, input_size, input_num
#     model.layer_list.append( CNN.Flattening(1, input_size=28, input_num=1, is_first=False))
    model.layer_list.append( CNN.Flattening(model.layer_list[len(model.layer_list)-1], input_size=model.layer_list[len(model.layer_list)-1].get_output_size(), input_num=5, is_first=False))
    
    #### First MLP
    # func, d_func, node_num, last_layer, is_first
#     model.layer_list.append( NN.Layer_vec(NN.ReLU, NN.d_ReLU, 100, model.layer_list[2], False) )
    
    #### Second MLP
    # func, d_func, node_num, last_layer, is_first
    model.layer_list.append( NN.Layer_vec(NN.ReLU, NN.d_ReLU, 64, model.layer_list[len(model.layer_list)-1], False) )
    
    #### Third MLP
    # func, d_func, node_num, last_layer, is_first
    model.layer_list.append( NN.Layer_vec(NN.ReLU, NN.d_ReLU, 10, model.layer_list[len(model.layer_list)-1], False) )
    
    #### Output Layer
    model.layer_list.append( NN.Softmax_Output(model.layer_list[len(model.layer_list)-1]) )
    print(model.layer_list)
            

In [ ]:
##############################################################
#                         MODEL block                        #
##############################################################

# with Notebook(): 
#     import NN_HW3_sm as NN
# lr_rate = 0.00009
epochs = 500

lr_rate = 0.0001
layer_nums = [20, 20, 20, 10]
# layer_nums = [train_data.shape[1]-1, 7, 1]
layer_input = preprocess_input(train_data[0])
test_m = NN.Model(layer_nums, construct_model, layer_input, lr_rate)

train_loss = []
test_loss = []
train_acc = []
test_acc = []
output_node_w = [[]]
output_node_w = np.array([test_m.get_output_w()])

for epoch in range(epochs):
    train_err_cnt = 0
    train_error = []
    print(epoch+1, "/", epochs, "epochs")
    train_progress = notebook.tqdm(total=len(train_data), desc="Training")
    test_progress = notebook.tqdm(total=len(test_data), desc="Testing")
    for i, input_data in enumerate(train_data):
#     for i, input_data in enumerate(train_data[0:100]):
#         print(i, "'s data !!!")
#         print(input_data)
        train_progress.update(1)
        p_data = preprocess_input(input_data)
        test_m.cal_network(p_data)
        ans_conf = test_m.get_result()
        ans = np.argmax(ans_conf)
        std = np.argmax(train_label[i])
        if ans != std:
            train_err_cnt += 1
        
        # Adjust model weights
        p_loss = test_m.adjust_model(train_label[i])
#         print(p_loss)
        train_error.append(p_loss)
#         print("ans_conf:　", ans_conf, "ans: ", ans, "train_label[i]:　", train_label[i], "std: ", std, "p_loss: ", p_loss)
#         print(test_m.get_output_w())
    output_node_w = np.append(output_node_w, [test_m.get_output_w()], 0)
        
    train_loss.append(np.average(train_error))
    train_acc.append(1 - train_err_cnt/len(train_data) )
    print("Train loss: ", np.average(train_error), ", acc = ", (1-train_err_cnt/len(train_data)) * 100, "%")
    test_err_cnt = 0
    test_error = []
    for i, input_data in enumerate(test_data):
#     for i, input_data in enumerate(test_data[0:10]):
        # print(i, "'s data !!!")
        test_progress.update(1)
        
        p_data = preprocess_input(input_data)
        test_m.cal_network(p_data)
        ans_conf = test_m.get_result()
        ans = np.argmax(ans_conf)
        std = np.argmax(test_label[i])
        if ans != std:
            test_err_cnt += 1
        
        
        p_loss = test_m.get_loss(test_label[i])
        test_error.append(p_loss)
        
        
    test_loss.append(np.average(test_error))
    test_acc.append(1 - test_err_cnt/len(test_data) )
    print("Test loss: ", np.average(test_error), ", acc = ", (1-test_err_cnt/len(test_data)) * 100, "%")
    print()
test_m.get_loss()

[<CNN_pool_class.ConvLayer object at 0x0000013C83F003C8>, <CNN_pool_class.AvgPooling object at 0x0000013C83F00390>, <CNN_pool_class.Flattening object at 0x0000013C83F00400>, <NN_HW3_sm.Layer_vec object at 0x0000013C83F00438>, <NN_HW3_sm.Layer_vec object at 0x0000013C800DF828>, <NN_HW3_sm.Softmax_Output object at 0x0000013C83F00518>]
1 / 500 epochs


Train loss:  0.8767066724962955 , acc =  71.90666666666668 %
Test loss:  0.33137246061443415 , acc =  89.34 %

2 / 500 epochs


Train loss:  0.2715977341248638 , acc =  91.92833333333333 %
Test loss:  0.2355274306648897 , acc =  92.58999999999999 %

3 / 500 epochs


Train loss:  0.21911746032103038 , acc =  93.56 %
Test loss:  0.196928325846238 , acc =  93.85 %

4 / 500 epochs


Train loss:  0.18729567797018148 , acc =  94.40666666666667 %
Test loss:  0.16963388344182112 , acc =  94.69999999999999 %

5 / 500 epochs


...

Train loss:  0.141561262637856 , acc =  95.77166666666666 %
Test loss:  0.13682878344057084 , acc =  95.86 %

8 / 500 epochs


Train loss:  0.13280241302496562 , acc =  96.00833333333333 %
Test loss:  0.13103523364361164 , acc =  96.00999999999999 %

9 / 500 epochs


Train loss:  0.12568415823033435 , acc =  96.22333333333334 %
Test loss:  0.1278662851194321 , acc =  96.07 %

10 / 500 epochs


Train loss:  0.11978891926339644 , acc =  96.38166666666666 %
Test loss:  0.12383984231454401 , acc =  96.2 %

11 / 500 epochs


Train loss:  0.11471503948630843 , acc =  96.575 %
Test loss:  0.12140329960631922 , acc =  96.3 %

12 / 500 epochs


In [ ]:
print("Train loss: ", (train_error))

In [ ]:
# write your code here
# print(output_node_w)
x= np.arange(0,len(output_node_w))
fig = plt.figure(figsize=(8,12))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
plt.subplot(2,1,1)
plt.plot(x, output_node_w[:,:,0])
plt.title('Weight Change')
plt.xlabel("Epoch") 
plt.ylabel("value") 


In [ ]:
# write your code here
x= np.arange(0,len(train_acc))
fig = plt.figure(figsize=(8,12))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
plt.subplot(2,1,1)
plt.plot(x, train_acc, color='Blue', label='Train')
plt.plot(x, test_acc, color='Orange', label='Validation')
plt.title('Model accuracy')
plt.xlabel("Epoch") 
plt.ylabel("Accuracy") 

plt.legend(loc='upper left', shadow=True) 

plt.subplot(2,1,2)
plt.plot(x, train_loss, color='Blue', label='Train')
plt.plot(x, test_loss, color='Orange', label='Validation')
plt.title('Model loss')
plt.xlabel("Epoch") 
plt.ylabel("Loss") 

plt.legend(loc='upper left', shadow=True) 